<a href="https://colab.research.google.com/github/tesseract314/DS-Unit-4-Sprint-3-Neural-Networks/blob/master/DS_Unit_4_Sprint_3_Neural_Nets_Sprint_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Networks Sprint Challenge

## 1) Define the following terms:

- Neuron
- Input Layer
- Hidden Layer
- Output Layer
- Activation
- Backpropagation

#### Neuron
A Neuron is a single, basic unit of computation in a neural network, also referred to as a node. It recieves information from the outside world or another Neuron and does computation to send to the next layer in the Nueral Network.

#### Input Layer
Inputs are information from the outside world that are given to the neural network. All inputs make up the Input Layer.

#### Hidden Layer
The Hidden Layer is made up of many hidden nodes, which have no connection to the outside world. Their purpose is to take information from the Input Layer, do caclulations on that information and pass the results to the the next Hidden Layer or Output Layer.

#### Output Layer
The Output Layer collects information from the Hidden Layers, does final calculations and transfers information from the neural network to the outside world.

#### Activation
Activation function of a neuron is a mathematical function that computes the output of that neuron.  Activation is good at introducing non-linearity to the outputs of neurons.

#### Backpropagation
Backpropagation is a calculus-intensive algorithm for updating weights in a neural network at the end of each epoch. Backpropagation is needed to improve the accuracy of a Neural Networks accuracy.


In [0]:
import numpy as np
import pandas as pd
import keras
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from keras.optimizers import Adadelta, SGD
from scipy import optimize

## 2) Create a perceptron class that can model the behavior of an AND gate. You can use the following table as your training data:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 1  | 1  | 1  | 1 |
| 1  | 0  | 1  | 0 |
| 0  | 1  | 1  | 0 |
| 0  | 0  | 1  | 0 |

In [0]:
# add data to arrays

X = np.array([
    [1, 1, 1],
    [1, 0, 1],
    [0, 1, 1],
    [0, 0, 1]
])

y = np.array([
    [1],
    [0],
    [0],
    [0]
])

In [0]:
# perceptron class
class Perceptron(object):
  def __init__(self, rate = 0.01, niter = 10):
    self.rate = rate
    self.niter = niter

  def fit(self, X, y):
    """Fit training data
    """

    # weights
    self.weight = np.zeros(1 + X.shape[1])

    # Number of misclassifications
    self.errors = []  # Number of misclassifications

    for i in range(self.niter):
      err = 0
      for xi, target in zip(X, y):
        delta_w = self.rate * (target - self.predict(xi))
        self.weight[1:] += delta_w * xi
        self.weight[0] += delta_w
        err += int(delta_w != 0.0)
      self.errors.append(err)
    return self

  def net_input(self, X):
    """Calculate net input"""
    return np.dot(X, self.weight[1:]) + self.weight[0]

  def predict(self, X):
    """Return class label after unit step"""
    return np.where(self.net_input(X) >= 0.5, 1, 0)

In [55]:
# perceptron instance with 0.1 learning rate and 1000 epochs
perc = Perceptron(0.1, 1000)
perc.fit(X, y) # fit model
perc.predict(X) # predict -- outputs correct

array([1, 0, 0, 0])

## 3) Implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights. 
- Your network must have one hidden layer. 
- You do not have to update weights via gradient descent. You can use something like the derivative of the sigmoid function to update weights.
- Train your model on the Heart Disease dataset from UCI:

[Github Dataset](https://github.com/ryanleeallred/datasets/blob/master/heart.csv)

[Raw File on Github](https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv)


In [27]:
df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [0]:
# separate into X and y
X = df.drop(columns='target').values
y = df[['target']].values

In [29]:
# see shapes
X.shape, y.shape

((303, 13), (303, 1))

In [0]:
class Neural_Network(object):
  def __init__(self):
    self.inputs = 13
    self.hiddenNodes = 3
    self.outputNodes = 1

    # Initlize Weights
    self.L1_weights = np.random.randn(self.inputs, self.hiddenNodes) 
    self.L2_weights = np.random.randn(self.hiddenNodes, self.outputNodes)

  def feed_forward(self, X):
    # Weighted sum between inputs and hidden layer
    self.hidden_sum = np.dot(X, self.L1_weights)
    # Activations of weighted sum
    self.activated_hidden = self.sigmoid(self.hidden_sum)
    # Weighted sum between hidden and output
    self.output_sum = np.dot(self.activated_hidden, self.L2_weights)
    # final activation of output
    self.activated_output = self.sigmoid(self.output_sum)
    return self.activated_output
    
  def sigmoid(self, s):
    return 1/(1+np.exp(-s))
  
  def sigmoidPrime(self, s): # derivative of sigmoid
    return s * (1 - s)
  
  def backward(self, X, y, o):
    # backward propgate through the network
    self.o_error = y - o # error in output
    self.o_delta = self.o_error*self.sigmoidPrime(o) # applying derivative of sigmoid to error

    self.z2_error = self.o_delta.dot(self.L2_weights.T) # z2 error: how much our hidden layer weights contributed to output error
    self.z2_delta = self.z2_error*self.sigmoidPrime(self.activated_hidden) # applying derivative of sigmoid to z2 error

    self.L1_weights += X.T.dot(self.z2_delta) # adjusting first set (input --> hidden) weights
    self.L2_weights += self.activated_hidden.T.dot(self.o_delta) # adjusting second set (hidden --> output) weights
    
  def train (self, X, y):
    o = self.feed_forward(X)
    self.backward(X, y, o)

In [0]:
# create instance and train
NN = Neural_Network()
NN.train(X, y)

## 4) Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy. 

- Use the Heart Disease Dataset (binary classification)
- Use an appropriate loss function for a binary classification task
- Use an appropriate activation function on the final layer of your network. 
- Train your model using verbose output for ease of grading.
- Use GridSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
- When hyperparameter tuning, show you work by adding code cells for each new experiment. 
- Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
- You must hyperparameter tune at least 5 parameters in order to get a 3 on this section.

In [57]:
# load data
df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [58]:
# separate into X and y
X = df.drop(columns='target').values
y = df['target'].values

X.shape, y.shape

((303, 13), (303,))

#### Baseline

In [67]:
# baseline model

# model function
def create_model():
  input_dim = X.shape[1]
  model = Sequential()
  model.add(Dense(12, input_dim=input_dim, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
  

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# define the grid search parameters
param_grid = {'batch_size': [50],
              'epochs': [50]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/50
202/202 [==============================] - 1s 3ms/step - loss: 5.1067 - acc: 0.6832
Epoch 2/50
202/202 [==============================] - 0s 48us/step - loss: 5.1067 - acc: 0.6832
Epoch 3/50
202/202 [==============================] - 0s 49us/step - loss: 5.1067 - acc: 0.6832
Epoch 4/50
202/202 [==============================] - 0s 44us/step - loss: 5.1067 - acc: 0.6832
Epoch 5/50
202/202 [==============================] - 0s 48us/step - loss: 5.1067 - acc: 0.6832
Epoch 6/50
202/202 [==============================] - 0s 42us/step - loss: 5.1067 - acc: 0.6832
Epoch 7/50
202/202 [==============================] - 0s 47us/step - loss: 5.1067 - acc: 0.6832
Epoch 8/50
202/202 [==============================] - 0s 44us/step - loss: 5.1067 - acc: 0.6832
Epoch 9/50
202/202 [==============================] - 0s 50us/step - loss: 5.1067 - acc: 0.6832
Epoch 10/50
202/202 [==============================] - 0s 50us/step - loss: 5.1067 - acc: 0.6832
Epoch 11/50
202/202 [===================

#### Tune Batch Size

In [68]:
# model function
def create_model():
  input_dim = X.shape[1]
  model = Sequential()
  model.add(Dense(12, input_dim=input_dim, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
  

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# define the grid search parameters
param_grid = {'batch_size': [30, 40, 50, 60, 80, 100],
              'epochs': [50]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/50
202/202 [==============================] - 0s 1ms/step - loss: 5.1067 - acc: 0.6832
Epoch 2/50
202/202 [==============================] - 0s 66us/step - loss: 5.1067 - acc: 0.6832
Epoch 3/50
202/202 [==============================] - 0s 59us/step - loss: 5.1067 - acc: 0.6832
Epoch 4/50
202/202 [==============================] - 0s 78us/step - loss: 5.1067 - acc: 0.6832
Epoch 5/50
202/202 [==============================] - 0s 61us/step - loss: 5.1067 - acc: 0.6832
Epoch 6/50
202/202 [==============================] - 0s 66us/step - loss: 5.1067 - acc: 0.6832
Epoch 7/50
202/202 [==============================] - 0s 67us/step - loss: 5.1067 - acc: 0.6832
Epoch 8/50
202/202 [==============================] - 0s 51us/step - loss: 5.1067 - acc: 0.6832
Epoch 9/50
202/202 [==============================] - 0s 57us/step - loss: 5.1067 - acc: 0.6832
Epoch 10/50
202/202 [==============================] - 0s 59us/step - loss: 5.1067 - acc: 0.6832
Epoch 11/50
202/202 [===================

#### Tune Epochs

In [69]:
# Tune epochs and keep best batch size

# model function
def create_model():
  input_dim = X.shape[1]
  model = Sequential()
  model.add(Dense(12, input_dim=input_dim, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
  

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# define the grid search parameters
param_grid = {'batch_size': [60],
              'epochs': [40, 50, 60, 70, 80]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/40
202/202 [==============================] - 2s 8ms/step - loss: 10.8913 - acc: 0.3168
Epoch 2/40
202/202 [==============================] - 0s 47us/step - loss: 10.8913 - acc: 0.3168
Epoch 3/40
202/202 [==============================] - 0s 48us/step - loss: 10.8913 - acc: 0.3168
Epoch 4/40
202/202 [==============================] - 0s 42us/step - loss: 10.8913 - acc: 0.3168
Epoch 5/40
202/202 [==============================] - 0s 50us/step - loss: 10.8913 - acc: 0.3168
Epoch 6/40
202/202 [==============================] - 0s 48us/step - loss: 10.8913 - acc: 0.3168
Epoch 7/40
202/202 [==============================] - 0s 46us/step - loss: 10.8913 - acc: 0.3168
Epoch 8/40
202/202 [==============================] - 0s 39us/step - loss: 10.8913 - acc: 0.3168
Epoch 9/40
202/202 [==============================] - 0s 45us/step - loss: 10.8913 - acc: 0.3168
Epoch 10/40
202/202 [==============================] - 0s 39us/step - loss: 10.8913 - acc: 0.3168
Epoch 11/40
202/202 [=========

#### Tune Optimization Algorithm

In [70]:
# create model
def create_model(optimizer='adam'):
  input_dim = X.shape[1]
  model = Sequential()
  model.add(Dense(12, input_dim=input_dim, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

# create KerasClassifier with best epochs and batch size
model = KerasClassifier(build_fn=create_model, epochs=70, batch_size=60, verbose=1)

# optimizer list
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

# param_grid
param_grid = dict(optimizer=optimizer)

# grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/70
202/202 [==============================] - 2s 12ms/step - loss: 10.8913 - acc: 0.3168
Epoch 2/70
202/202 [==============================] - 0s 63us/step - loss: 10.8913 - acc: 0.3168
Epoch 3/70
202/202 [==============================] - 0s 47us/step - loss: 10.8913 - acc: 0.3168
Epoch 4/70
202/202 [==============================] - 0s 53us/step - loss: 10.8913 - acc: 0.3168
Epoch 5/70
202/202 [==============================] - 0s 48us/step - loss: 10.8913 - acc: 0.3168
Epoch 6/70
202/202 [==============================] - 0s 49us/step - loss: 10.8913 - acc: 0.3168
Epoch 7/70
202/202 [==============================] - 0s 49us/step - loss: 10.8913 - acc: 0.3168
Epoch 8/70
202/202 [==============================] - 0s 46us/step - loss: 10.8913 - acc: 0.3168
Epoch 9/70
202/202 [==============================] - 0s 51us/step - loss: 10.8913 - acc: 0.3168
Epoch 10/70
202/202 [==============================] - 0s 54us/step - loss: 10.8913 - acc: 0.3168
Epoch 11/70
202/202 [========

#### Tune Learning Rate

In [73]:
# create model
def create_model(learn_rate=0.01):
  input_dim = X.shape[1]
  model = Sequential()
  model.add(Dense(12, input_dim=input_dim, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  
  # use best optimizer -- adadelta
  optimizer = Adadelta(lr=learn_rate)
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

# create KerasClassifier with best epochs and batch size
model = KerasClassifier(build_fn=create_model, epochs=70, batch_size=60, verbose=1)

# optimizer list
learn_rate = [0.0001, 0.001, 0.01, .1, .2]

# param_grid
param_grid = dict(learn_rate=learn_rate)

# grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# print results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")
  

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/70
202/202 [==============================] - 3s 15ms/step - loss: 5.1067 - acc: 0.6832
Epoch 2/70
202/202 [==============================] - 0s 54us/step - loss: 5.1067 - acc: 0.6832
Epoch 3/70
202/202 [==============================] - 0s 51us/step - loss: 5.1067 - acc: 0.6832
Epoch 4/70
202/202 [==============================] - 0s 49us/step - loss: 5.1067 - acc: 0.6832
Epoch 5/70
202/202 [==============================] - 0s 49us/step - loss: 5.1067 - acc: 0.6832
Epoch 6/70
202/202 [==============================] - 0s 50us/step - loss: 5.1067 - acc: 0.6832
Epoch 7/70
202/202 [==============================] - 0s 54us/step - loss: 5.1067 - acc: 0.6832
Epoch 8/70
202/202 [==============================] - 0s 55us/step - loss: 5.1067 - acc: 0.6832
Epoch 9/70
202/202 [==============================] - 0s 55us/step - loss: 5.1067 - acc: 0.6832
Epoch 10/70
202/202 [==============================] - 0s 53us/step - loss: 5.1067 - acc: 0.6832
Epoch 11/70
202/202 [==================

#### Tune Momentum

In [80]:
# create model
def create_model(learn_rate=0.001, momentum=0): # use best learning rate
  input_dim = X.shape[1]
  model = Sequential()
  model.add(Dense(12, input_dim=input_dim, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  optimizer = SGD(lr=learn_rate, momentum=momentum) # switch to SGD optimizer for momentum param tuning
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

# create KerasClassifier with best epochs and batch size
model = KerasClassifier(build_fn=create_model, epochs=70, batch_size=60, verbose=1)

# optimizer list
momentum = [0.0, 0.2, 0.4, 0.6, 0.8]
learn_rate= [0.001]

# param_grid
param_grid = dict(learn_rate=learn_rate, momentum=momentum)

# grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# print results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/70
202/202 [==============================] - 4s 22ms/step - loss: 5.1067 - acc: 0.6832
Epoch 2/70
202/202 [==============================] - 0s 60us/step - loss: 5.1067 - acc: 0.6832
Epoch 3/70
202/202 [==============================] - 0s 50us/step - loss: 5.1067 - acc: 0.6832
Epoch 4/70
202/202 [==============================] - 0s 56us/step - loss: 5.1067 - acc: 0.6832
Epoch 5/70
202/202 [==============================] - 0s 53us/step - loss: 5.1067 - acc: 0.6832
Epoch 6/70
202/202 [==============================] - 0s 58us/step - loss: 5.1067 - acc: 0.6832
Epoch 7/70
202/202 [==============================] - 0s 65us/step - loss: 5.1067 - acc: 0.6832
Epoch 8/70
202/202 [==============================] - 0s 59us/step - loss: 5.1067 - acc: 0.6832
Epoch 9/70
202/202 [==============================] - 0s 53us/step - loss: 5.1067 - acc: 0.6832
Epoch 10/70
202/202 [==============================] - 0s 57us/step - loss: 5.1067 - acc: 0.6832
Epoch 11/70
202/202 [==================